In [ ]:
!pip install mediapipe

In [ ]:
import os
import cv2
import mediapipe as mp
import csv

In [ ]:
def process_video(video_path, csv_writer, condition_severity, pose, frame_id_start=0):
    cap = cv2.VideoCapture(video_path)
    frame_id = frame_id_start
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        if results.pose_landmarks is not None:
            keypoints = [[landmark.x, landmark.y] for landmark in results.pose_landmarks.landmark]
        else:
            keypoints = [[0, 0] for _ in range(48)]  # Fill with zeros if no keypoints detected
        row = [frame_id] + [coord for point in keypoints for coord in (point[0], point[1])] + [condition_severity]
        csv_writer.writerow(row)
        frame_id += 1
    cap.release()
    return frame_id


In [ ]:
def process_directory_structure(data_directory):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

    for root, dirs, files in os.walk(data_directory):
        if files:  # Check if there are files in the current directory
            metadata_parts = root.split(os.sep)[-1].split('_')
            condition = metadata_parts[0]
            if len(metadata_parts) > 1:
                severity = metadata_parts[1]
            else:
                severity = 'NA'
            condition_severity = f'{condition}_{severity}'
            csv_filename = f'{condition}_{severity}_keypoints.csv'
            csv_path = os.path.join(root, csv_filename)
            with open(csv_path, 'w', newline='') as file:
                writer = csv.writer(file)
                header = ['frame_id'] + [f'point_{i}x, point_{i}_y' for i in range(33)] + ['target']
                writer.writerow(header)
                frame_id = 0
                for filename in files:
                    if filename.endswith('.MOV'):  # Check if the file is a MOV video
                        video_path = os.path.join(root, filename)
                        frame_id = process_video(video_path, writer, condition_severity, pose, frame_id)  # Process each video

    pose.close()

In [ ]:
data_directory = r'Data\44pfnysy89-1\KOA-PD-NM'

In [ ]:
process_directory_structure(data_directory)